In [9]:
import os
import gmaps
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json

from api_keys import g_key
gmaps.configure(api_key=g_key)



In [10]:
#Load csv
cities = pd.read_csv("newcities.csv", encoding="utf-8")
cities.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Severo-Kuril'sk,84,RU,1655902277,96,50.6789,156.1250,44.64,12.46
1,1,Maragogi,100,BR,1655902200,90,-9.0122,-35.2225,75.69,15.50
2,2,Busselton,98,AU,1655902119,77,-33.6500,115.3333,64.42,12.82
3,3,Sinnamary,99,GF,1655902126,78,5.3833,-52.9500,79.54,5.10
4,4,Warrington,20,GB,1655902307,54,53.3925,-2.5802,77.86,11.50


In [3]:
#Humidity Heatmap

humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity= 10,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Create new DataFrame fitting weather criteria

narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()




,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,5,Mikuni,0,JP,1655902203,92,36.2171,136.1519,75.11,7.23
1,17,Mount Darwin,0,ZW,1655902311,31,-16.7725,31.5838,73.99,9.26
2,25,Plouzané,0,FR,1655902313,66,48.3833,-4.6167,75.25,9.22
3,26,Saint-Philippe,0,RE,1655902137,68,-21.3585,55.7679,71.19,10.45
4,71,Cape Town,0,ZA,1655902120,59,-33.9258,18.4232,71.38,6.91


In [6]:
#hotel map

hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,5,Mikuni,0,JP,1655902203,92,36.2171,136.1519,75.11,7.23,Sakai
1,17,Mount Darwin,0,ZW,1655902311,31,-16.7725,31.5838,73.99,9.26,Mount Darwin
2,25,Plouzané,0,FR,1655902313,66,48.3833,-4.6167,75.25,9.22,Brest
3,26,Saint-Philippe,0,RE,1655902137,68,-21.3585,55.7679,71.19,10.45,Saint-Philippe
4,71,Cape Town,0,ZA,1655902120,59,-33.9258,18.4232,71.38,6.91,Cape Town


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [8]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))